# Proyecto Clase - Análisis de homogeneidad covariables formadoras de suelo
## Procesamiento inicial de datos
1. Reproyectar el DEM, la imagen Sentinel 2 y los Índices Calculados de Curití con el crs del límite de la cuenca

In [1]:
import sys
sys.path.append("/notebooks/Proyecto_Clase/src")
import importlib
import data_preprocessing
importlib.reload(data_preprocessing)

from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
import os
import glob
from joblib import Parallel, delayed

In [17]:
ruta_shp = "/notebooks/Insumos_Proyecto_Clase/"
ruta_ras = "/notebooks/Insumos_Proyecto_Clase/"

nombres_shp = {
    "geom": "Geomorfologia.shp",
    "geol": "Geologia.shp",
    "clima": "Clima_IGAC.shp",
    "limBuf": "Limite_Buff_500.shp"
}

nombres_raster = {
    "sentinel2": "Sentinel2.tif",
    "dem": "Dem_Cortado_Buff.tif",
    "indices_S2": "Indices_curiti_Sentinel2.tif"
}

insumos = data_preprocessing.carga_shp_raster_proj(ruta_shp, nombres_shp, ruta_ras, nombres_raster)
geom_shp = insumos["shapefiles"]["geom"]
geol_shp = insumos["shapefiles"]["geol"]
clim_shp = insumos["shapefiles"]["clima"]
limBuf_shp = insumos["shapefiles"]["limBuf"]

sentinel2 = insumos["rasters"]["sentinel2"]
Dem_cortado = insumos["rasters"]["dem"]
indices_s2 = insumos["rasters"]["indices_S2"]

✅Shapefiles cargados geom

✅Shapefiles cargados geol

✅Shapefiles cargados clima

✅Shapefiles cargados limBuf

✅CRS de los Shapefiles cargados EPSG:9377



Datos del raster cargado sentinel2
 {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 1741, 'height': 1858, 'count': 8, 'crs': CRS.from_epsg(4326), 'transform': Affine(9.039999999999594e-05, 0.0, -73.13438320910319,
       0.0, -9.039999999999999e-05, 6.6712409126466605)}


⚠️ CRS del raster "sentinel2" no coincide con el CRS de los shapefiles. Reproyectando...


✅ Raster reproyectado y guardado en: /notebooks/Insumos_Proyecto_Clase/Sentinel2_Reproyectado.tif




Datos del raster cargado dem
 {'driver': 'GTiff', 'dtype': 'int16', 'nodata': 32767.0, 'width': 1453, 'height': 1552, 'count': 1, 'crs': CRS.from_wkt('PROJCS["MAGNA-SIRGAS / Colombia Bogota zone",GEOGCS["MAGNA-SIRGAS",DATUM["Marco_Geocentrico_Nacional_de_Referencia",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTH

## Conversion a Raster de capas shape y resampleo
1. Carga los shapes
2. Homogeniza sistema coordenado a EPSG 9377
3. Rasteriza por las columnas del shape
4. Ingresa todos los rasters, incluyendo los indices sentinel y el dem con sus parámetros
5. Resamplea los rasters a 20m

In [31]:
import sys
sys.path.append("/notebooks/Proyecto_Clase/src")
import raster_processing
importlib.reload(raster_processing)

# Define las rutas y los nombres de los archivos
ruta_shp = "/notebooks/Insumos_Proyecto_Clase/"
ruta_ras = "/notebooks/Insumos_Proyecto_Clase/"
nombres_shp = {
    "geom": "Geomorfologia_r.shp",
    "geol": "Geologia_r.shp",
    "clima": "Clima_IGAC_r.shp"
}
columna_rasterizada = {
    "geom": {"Simbolo": {"raster": "geom_raster.tif", "output": "output_geom.tif"}},
    "geol": {"Nomenclat": {"raster": "geol_raster.tif", "output": "output_geol.tif"}},
    "clima": {"Piso Termico": {"raster": "clima_raster.tif", "output": "output_clima_piso.tif"},
              "Clase_Clim": {"raster": "clima_clase_raster.tif", "output": "output_clima_clase.tif"}}
}
}

# Llama a la función para ejecutar el procesamiento
raster_processing.rasterizacion(ruta_shp, ruta_ras, nombres_shp, nombres_raster, columna_rasterizada)

Proceso de rasterización y resampleo completado.


## Corte por Geoformas
1. Carga las imágenes
2. Corta las imágenes iterando por cada símbolo de geoforma

In [32]:
import sys
sys.path.append("/notebooks/Proyecto_Clase/src")
import clip_rasters
importlib.reload(clip_rasters)

# Define las rutas y los nombres de los archivos
ruta_shp = "/notebooks/Insumos_Proyecto_Clase/"
ruta_ras = "/notebooks/Insumos_Proyecto_Clase/"
nombres_shp = {
    "geom": "Geomorfologia_clip_igac.shp"
}
nombres_raster = {
    "dem": "Dem_Cortado_Buff_Reproyectado.tif",
    "indices_S2": "Indices_curiti_Sentinel2_Reproyectado.tif",
    "climaP": "output_clima_piso.tif",
    "climaC": "output_clima_clase.tif",
    "geol" : "output_geol.tif"
}

output_dir = "/notebooks/Insumos_Proyecto_Clase/Raster_Cortados/"

# Llama a la función para cortar los rasters por geoforma
clip_rasters.cortar_rasters_por_geoformas(ruta_shp, ruta_ras, nombres_shp, nombres_raster, output_dir)

Procesando dem para geoforma MCRs
Recorte de dem para geoforma MCRs fue exitoso
Raster cortado guardado como: /notebooks/Insumos_Proyecto_Clase/Raster_Cortados/DEM_MCRs.tif
Procesando indices_S2 para geoforma MCRs
Recorte de indices_S2 para geoforma MCRs fue exitoso
Raster cortado guardado como: /notebooks/Insumos_Proyecto_Clase/Raster_Cortados/Ind_MCRs.tif
Procesando climaP para geoforma MCRs
Recorte de climaP para geoforma MCRs fue exitoso
Raster cortado guardado como: /notebooks/Insumos_Proyecto_Clase/Raster_Cortados/ClimaP_MCRs.tif
Procesando climaC para geoforma MCRs
Recorte de climaC para geoforma MCRs fue exitoso
Raster cortado guardado como: /notebooks/Insumos_Proyecto_Clase/Raster_Cortados/ClimaC_MCRs.tif
Procesando geol para geoforma MCRs
Recorte de geol para geoforma MCRs fue exitoso
Raster cortado guardado como: /notebooks/Insumos_Proyecto_Clase/Raster_Cortados/geol_MCRs.tif
Procesando dem para geoforma MVV
Recorte de dem para geoforma MVV fue exitoso
Raster cortado guardad